In [1]:
file = open("nietzsche.txt", "r") 
data = file.read()


In [2]:
int_data = []
dictionary = {}
reverse_dictionary = {}
dic_len = 0
for i in range(len(data)):
    if data[i] in dictionary:
        int_data.append(dictionary[data[i]])
    else:
        dictionary[data[i]] = dic_len
        reverse_dictionary[dic_len] = data[i]
        dic_len += 1
        

In [3]:
import numpy as np
def to_one_hat(data, dic_len):
    out = np.zeros((len(data), dic_len))
    for i, d in enumerate(data):
        out[i, d] = 1
    return out

clean_data = to_one_hat(int_data, dic_len)

In [4]:
len_seq = 40
self_overlap = 3

last_data = np.zeros((len(data) // (len_seq - self_overlap), len_seq, dic_len))
for i in range(0, len(clean_data) - len_seq, len_seq - self_overlap):
    last_data[i // (len_seq - self_overlap),:, :] = clean_data[i: i + len_seq , :]
    

In [5]:
print(reverse_dictionary)

{0: 'P', 1: 'R', 2: 'E', 3: 'F', 4: 'A', 5: 'C', 6: '\n', 7: 'S', 8: 'U', 9: 'O', 10: 'I', 11: 'N', 12: 'G', 13: ' ', 14: 't', 15: 'h', 16: 'a', 17: 'T', 18: 'r', 19: 'u', 20: 'i', 21: 's', 22: 'w', 23: 'o', 24: 'm', 25: 'n', 26: '-', 27: 'e', 28: '?', 29: 'g', 30: 'd', 31: 'f', 32: 'p', 33: 'c', 34: 'l', 35: ',', 36: 'y', 37: 'v', 38: 'b', 39: 'k', 40: ';', 41: '!', 42: '.', 43: 'B', 44: 'z', 45: 'W', 46: 'H', 47: ':', 48: '(', 49: 'j', 50: ')', 51: '"', 52: 'V', 53: 'L', 54: "'", 55: 'D', 56: 'Y', 57: 'K', 58: 'q', 59: 'M', 60: 'x', 61: 'J', 62: '1', 63: '8', 64: '5', 65: '2', 66: '3', 67: '_', 68: '4', 69: '6', 70: '7', 71: '9', 72: '0', 73: 'Q', 74: 'X', 75: '[', 76: ']', 77: 'Z', 78: 'ä', 79: '=', 80: 'æ', 81: 'ë', 82: 'é', 83: 'Æ'}


In [10]:
import tensorflow as tf
tf.reset_default_graph()

n_hidden = 128
gru = tf.contrib.rnn.GRUCell(n_hidden)
feature_size = dic_len
learning_rate = 0.01
batch_size = 50

input_data = tf.placeholder(dtype=tf.float32, shape=[None, None, feature_size])
target_data = tf.placeholder(dtype=tf.float32, shape=[None, None, feature_size])


initial_state = gru.zero_state(1, dtype=tf.float32)

output, state = tf.nn.dynamic_rnn(gru, input_data, dtype = tf.float32)

w = tf.Variable(tf.random_normal([n_hidden, feature_size]))
b = tf.Variable(tf.random_normal([feature_size]))
                
output_reshaped = tf.reshape(output, [-1 , n_hidden])

logits = tf.matmul(output_reshaped, w) + b

last_output = tf.nn.softmax(tf.reshape(logits, tf.shape(input_data)))

preds = tf.argmax(last_output, 2)

loss = tf.reduce_mean((target_data - last_output) * (target_data - last_output))
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(loss)

init = tf.global_variables_initializer()



In [11]:
saver = tf.train.Saver()

In [12]:
#just some config for not getting whole server
gpu_options = tf.GPUOptions(allow_growth=True)
config = tf.ConfigProto(gpu_options=gpu_options)

epochs_num = 30
sess = tf.Session(config = config) 
sess.run(init)
for e in range(epochs_num):
    total_loss = 0
    n_epochs = len(last_data)//batch_size
    for i in range(n_epochs):
        batch = last_data[batch_size * i: batch_size * (i+1), :, :]
        X_batch, Y_batch = batch[:, 0:len_seq -1, :], batch[:, 1:len_seq, :]
        train_loss,_,train_preds = sess.run([loss, optimizer, preds]
                                            ,feed_dict = {input_data : X_batch, target_data : Y_batch})
        total_loss += train_loss
    print("epoch {}, train loss = {}".format(e, total_loss))
        
    



epoch 0, train loss = 3.679473824799061
epoch 1, train loss = 3.0949996849521995
epoch 2, train loss = 2.90929121337831
epoch 3, train loss = 2.784828462637961
epoch 4, train loss = 2.682895469944924
epoch 5, train loss = 2.606905061751604
epoch 6, train loss = 2.5494548538699746
epoch 7, train loss = 2.5061772358603776
epoch 8, train loss = 2.469462802633643
epoch 9, train loss = 2.439052229747176
epoch 10, train loss = 2.41227811248973
epoch 11, train loss = 2.3882595845498145
epoch 12, train loss = 2.3666033474728465
epoch 13, train loss = 2.3481959919445217
epoch 14, train loss = 2.3305799239315093
epoch 15, train loss = 2.314336543902755
epoch 16, train loss = 2.300010702572763
epoch 17, train loss = 2.287126758135855
epoch 18, train loss = 2.2753186910413206
epoch 19, train loss = 2.2643421958200634
epoch 20, train loss = 2.25411623576656
epoch 21, train loss = 2.245194101240486
epoch 22, train loss = 2.236884409096092
epoch 23, train loss = 2.2287738197483122
epoch 24, train los

In [20]:
import datetime
dt = datetime.datetime.now().strftime("%Y-%m-%d-%H")
print(dt)
saver.save(sess, 'Models/model-{}/model'.format(dt))

2017-11-28-12


'Models/model-2017-11-28-12/model'

In [21]:
def to_one_hat(x, dic_len):
    out = np.zeros(dic_len)
    out[x] = 1
    return out

test_data = last_data[10000:10001]

for i in range(len_seq):
    print(reverse_dictionary[np.argmax(test_data[0,i])], end = "")
for i in range( 400):
    test_output, test_preds = sess.run([ last_output, preds] ,feed_dict = {input_data : test_data})
    
    picked_word = np.random.choice(dic_len, 1, p=test_output[0, -1, :])[0]
    print(reverse_dictionary[picked_word],end = "")

    test_data[0, 0:len_seq - 1, :] = test_data[0, 1:len_seq, :]
    test_data[0, len_seq - 1, :] = to_one_hat(picked_word, dic_len)

 is a regular cult of
suffering. The UNMON of spotker tomances sai's saintiathols ineared in the owmost tamoncessa. 4undred to SOMPer, his heart of Yemanciationtely de=Ægow bi: dever besinessible, Aætiondomening aéceriom all conceéed the maintenction and
"False and withever happin thinder these is helHest at2o
traced, in sacallfy fa_ being of a1ST
A) onves the mot'. ëontinequal powentryElTron of sfactianity of power, the smilarian
self2

In [22]:
sess.close()